In [59]:
#import libraries

import numpy as np
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from sklearn.metrics import r2_score
import data_ML as ML

np.set_printoptions(suppress=True)

In [60]:
data = []
df = ML.getdata(False)
df.reset_index()

df_calls = df[df.optionType == 1].drop(columns='optionType')
df_american_calls = df_calls[df_calls.method == 1].drop(columns='method')
df_american_calls
df_american_calls = df_american_calls[df_american_calls.lastPrice > 5]
#training and test datasets
y = np.asarray(df_american_calls[['lastPrice']])
df_american_calls = df_american_calls.drop(columns='lastPrice')
df_american_calls
y
df


,optionType,strike,maturity,S0,sigma,returns,method,lastPrice
0,1,50.0,0.057534,176.417572,0.332644,0.256235,1,125.70
1,1,75.0,0.057534,176.417572,0.332644,0.256235,1,100.21
2,0,95.0,0.057534,176.417572,0.332644,0.256235,1,0.01
3,0,240.0,0.057534,176.417572,0.332644,0.256235,1,64.45
4,1,225.0,0.076712,176.417572,0.332644,0.256235,1,0.01
...,...,...,...,...,...,...,...,...
13214,0,55.0,0.575342,20.190001,1.241391,0.043145,0,33.25
13215,1,12.5,0.649315,20.190001,1.241391,0.043145,0,9.05
13216,1,31.0,0.649315,20.190001,1.241391,0.043145,0,2.30
13217,0,45.0,0.687671,16.090000,1.240779,-0.003441,0,24.90


In [61]:
for line in range(len(df_american_calls)):
    data_tmp = df_american_calls.iloc[line]
    data.append(data_tmp)

In [62]:
data = np.asarray(data)
X = data.reshape((data.shape[0], data.shape[1], 1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
y

array([[125.7 ],
       [100.21],
       [ 50.77],
       ...,
       [245.4 ],
       [217.25],
       [226.7 ]])

In [63]:
#ANN with four layers, 10 neurons each
#activation function: ReLU

# model = Sequential()
# model.add(Dense(64, input_dim=5, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(1))

# model = Sequential()
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(X.shape[1], 1)))
# model.add(MaxPooling1D(pool_size=2, strides=3))
# model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
# model.add(MaxPooling1D(pool_size=2, strides=3))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1))

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(X.shape[1], 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2, strides=3))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2, strides=3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1))

from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=15, verbose=1)

# from keras.optimizers import SGD, RMSprop

# sgd = SGD(lr=0.01, momentum=0.9)
# rmsprop = RMSprop(lr=0.001)

# model.compile(optimizer=rmsprop, loss='mse')
model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_split=0.2)



Epoch 1/200
18/18 [==============================] - 0s 5ms/step - loss: 20854.6406 - val_loss: 18018.6523
Epoch 2/200
18/18 [==============================] - 0s 2ms/step - loss: 20411.3242 - val_loss: 17605.3535
Epoch 3/200
18/18 [==============================] - 0s 2ms/step - loss: 20072.9434 - val_loss: 17042.6836
Epoch 4/200
18/18 [==============================] - 0s 2ms/step - loss: 19690.6777 - val_loss: 16667.2871
Epoch 5/200
18/18 [==============================] - 0s 2ms/step - loss: 19328.9199 - val_loss: 16213.5176
Epoch 6/200
18/18 [==============================] - 0s 2ms/step - loss: 18990.9121 - val_loss: 15616.7178
Epoch 7/200
18/18 [==============================] - 0s 2ms/step - loss: 18596.2949 - val_loss: 14935.6348
Epoch 8/200
18/18 [==============================] - 0s 2ms/step - loss: 18201.2148 - val_loss: 14582.7266
Epoch 9/200
18/18 [==============================] - 0s 2ms/step - loss: 17726.5254 - val_loss: 14308.0752
Epoch 10/200
18/18 [=================

In [71]:
#prediction
import pandas as pd
y_pred = model.predict(X_test)


db = pd.DataFrame({'prediction':y_pred.tolist(),'real':y_test.tolist()})
error = (y_pred - y_test)/y_test
db = db.assign(error = error)
mse = np.average(error**2)
print(mse)
db




15/15 [==============================] - 0s 504us/step
0.0966536766331705


,prediction,real,error
0,[218.80767822265625],[212.75],0.028473
1,[115.04769134521484],[113.0],0.018121
2,[188.63858032226562],[179.0],0.053847
3,[177.04727172851562],[182.68],-0.030834
4,[178.23385620117188],[174.8],0.019644
...,...,...,...
471,[67.39795684814453],[65.28],0.032444
472,[47.090789794921875],[48.0],-0.018942
473,[32.1861686706543],[35.37],-0.090015
474,[57.99631118774414],[57.83],0.002876
